In [163]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform,randint
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# from torch import 

In [2]:
train = pd.read_csv("train.csv")
submission = pd.read_csv("gender_submission.csv")
test = pd.read_csv("test.csv")
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
nan_counts = train.isna().sum()
nan_counts

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [25]:
train2 = train[["PassengerId", "Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]]
train2

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000
887,888,1,1,female,19.0,0,0,30.0000
888,889,0,3,female,NaN,1,2,23.4500
889,890,1,1,male,26.0,0,0,30.0000


In [26]:
train2.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [27]:
train2 = pd.get_dummies(train2,drop_first=True)
# train2.drop(columns=["Sex_female"],inplace=True)
train2

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,1,0,3,22.0,1,0,7.2500,True
1,2,1,1,38.0,1,0,71.2833,False
2,3,1,3,26.0,0,0,7.9250,False
3,4,1,1,35.0,1,0,53.1000,False
4,5,0,3,35.0,0,0,8.0500,True
...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000,True
887,888,1,1,19.0,0,0,30.0000,False
888,889,0,3,NaN,1,2,23.4500,False
889,890,1,1,26.0,0,0,30.0000,True


In [34]:
train2.corr()["Survived"]

PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
Age           -0.077221
SibSp         -0.035322
Parch          0.081629
Fare           0.257307
Sex_male      -0.543351
Name: Survived, dtype: float64

In [42]:
train3 = train2.dropna()
# train3.isna().sum()
train3

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,1,0,3,22.0,1,0,7.2500,True
1,2,1,1,38.0,1,0,71.2833,False
2,3,1,3,26.0,0,0,7.9250,False
3,4,1,1,35.0,1,0,53.1000,False
4,5,0,3,35.0,0,0,8.0500,True
...,...,...,...,...,...,...,...,...
885,886,0,3,39.0,0,5,29.1250,False
886,887,0,2,27.0,0,0,13.0000,True
887,888,1,1,19.0,0,0,30.0000,False
889,890,1,1,26.0,0,0,30.0000,True


In [52]:
col = train3.columns
kek = col[2:]
train3[kek]

,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,3,22.0,1,0,7.2500,True
1,1,38.0,1,0,71.2833,False
2,3,26.0,0,0,7.9250,False
3,1,35.0,1,0,53.1000,False
4,3,35.0,0,0,8.0500,True
...,...,...,...,...,...,...
885,3,39.0,0,5,29.1250,False
886,2,27.0,0,0,13.0000,True
887,1,19.0,0,0,30.0000,False
889,1,26.0,0,0,30.0000,True


In [92]:
X = train2[col[2:]]
y = train2[col[1]]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)



In [93]:
xgbmodel = xgb.XGBClassifier(n_estimators=743,learning_rate=0.01)
xgbmodel.fit(X_train,y_train)
y_pred = xgbmodel.predict(X_test)
acc = accuracy_score(y_pred,y_test)
acc

0.8379888268156425

In [102]:
X = train3[col[2:]]
y = train3[col[1]]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

rf_model = RandomForestClassifier(n_estimators=1000,random_state=42)
rf_model.fit(X_train,y_train)
rf_pred = rf_model.predict(X_test)
rf_acc = accuracy_score(rf_pred, y_test)
rf_acc

0.7832167832167832

In [107]:
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_acc = accuracy_score(lr_pred, y_test)
lr_acc

0.7482517482517482

In [110]:
train2.duplicated().sum()

0

In [205]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [221]:
cols = train.columns
train4 = train.iloc[:,[1,2,4,5,6,7,9,11]]
test4 = test.iloc[:,[1,3,4,5,6,7,8,10]]


In [222]:
test4

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,1,34.5,0,0,152,7.8292,1
1,3,0,47.0,1,0,221,7.0000,2
2,2,1,62.0,0,0,73,9.6875,1
3,3,1,27.0,0,0,147,8.6625,2
4,3,0,22.0,1,1,138,12.2875,2
...,...,...,...,...,...,...,...,...
413,3,1,NaN,0,0,267,8.0500,2
414,1,0,39.0,0,0,324,108.9000,0
415,3,1,38.5,0,0,346,7.2500,2
416,3,1,NaN,0,0,220,8.0500,2


In [223]:
train4["Age"].fillna(train4["Age"].median(),inplace=True)
test4["Age"].fillna(test4["Age"].median(),inplace=True)
test4['Fare'].fillna(test['Fare'].median(), inplace=True)

train4.isna().sum()
test4.isna().sum()


C:\Users\casov\AppData\Local\Temp\ipykernel_3364\1764840222.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train4["Age"].fillna(train4["Age"].median(),inplace=True)
C:\Users\casov\AppData\Local\Temp\ipykernel_3364\1764840222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train4["Age"].fillna(train4["Age"].median(),inplace=True)
C:\Us

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [224]:
train4["Embarked"].fillna(train4["Embarked"].mode()[0],inplace=True)
train4["Ticket"] = train["Ticket"]

C:\Users\casov\AppData\Local\Temp\ipykernel_3364\2619832835.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train4["Embarked"].fillna(train4["Embarked"].mode()[0],inplace=True)
C:\Users\casov\AppData\Local\Temp\ipykernel_3364\2619832835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train4["Embarked"].fillna(train4["Embarked"].mode()[

In [225]:
train4.duplicated().sum()

16

In [226]:
train4.drop_duplicates(inplace=True)

C:\Users\casov\AppData\Local\Temp\ipykernel_3364\1839749157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train4.drop_duplicates(inplace=True)


In [227]:
train4.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Ticket      0
dtype: int64

In [228]:
le = LabelEncoder()
for col in train4.columns:
    if train4[col].dtype=="object":
        train4[col] = le.fit_transform(train4[col])
        
for col in test4.columns:
  if test4[col].dtype=='object':
    test4[col]=le.fit_transform(test4[col])


C:\Users\casov\AppData\Local\Temp\ipykernel_3364\1466396821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train4[col] = le.fit_transform(train4[col])
C:\Users\casov\AppData\Local\Temp\ipykernel_3364\1466396821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train4[col] = le.fit_transform(train4[col])
C:\Users\casov\AppData\Local\Temp\ipykernel_3364\1466396821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [229]:
test4.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,1,34.5,0,0,152,7.8292,1
1,3,0,47.0,1,0,221,7.0000,2
2,2,1,62.0,0,0,73,9.6875,1
3,3,1,27.0,0,0,147,8.6625,2
4,3,0,22.0,1,1,138,12.2875,2


In [230]:
train4

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Ticket
0,0,3,1,22.0,1,0,7.2500,2,523
1,1,1,0,38.0,1,0,71.2833,0,596
2,1,3,0,26.0,0,0,7.9250,2,669
3,1,1,0,35.0,1,0,53.1000,2,49
4,0,3,1,35.0,0,0,8.0500,2,472
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,2,101
887,1,1,0,19.0,0,0,30.0000,2,14
888,0,3,0,28.0,1,2,23.4500,2,675
889,1,1,1,26.0,0,0,30.0000,0,8


In [249]:
# X = train4.iloc[:,1:]
# y = train4.iloc[:,0]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
X_train = train4.iloc[:, 1:]
y_train = train4.iloc[:, 0]
X_test = test4.iloc[:,0:]
X_test.drop(columns=["Ticket"],inplace=True)
X_test["Ticket"] = test4["Ticket"]
y_test = pd.read_csv("gender_submission.csv")
y_test.drop(['PassengerId'],axis=1,inplace=True)
y_test = y_test.iloc[:,:]
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
lr_pred = lr_model.predict(X_test)
lr_acc = accuracy_score(lr_pred, y_test)
lr_acc

c:\Users\casov\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9641148325358851

In [243]:
X_test


,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,1,34.5,0,0,152,7.8292,1
1,3,0,47.0,1,0,221,7.0000,2
2,2,1,62.0,0,0,73,9.6875,1
3,3,1,27.0,0,0,147,8.6625,2
4,3,0,22.0,1,1,138,12.2875,2
...,...,...,...,...,...,...,...,...
413,3,1,27.0,0,0,267,8.0500,2
414,1,0,39.0,0,0,324,108.9000,0
415,3,1,38.5,0,0,346,7.2500,2
416,3,1,27.0,0,0,220,8.0500,2


In [247]:
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [250]:

pred = pd.DataFrame({
    "PassengerId" : submission['PassengerId'],
    "Survived" : lr_pred
})
pred.to_csv('result.csv', index=False)

In [133]:


# Загрузка данных


# Выбор признаков (исключите 'Age' из признаков)
features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']  # Примеры признаков
df = pd.get_dummies(train[features + ['Age']], drop_first=True)

# Разделение данных на обучающую и тестовую выборки
train_data = df[df['Age'].notna()]
test_data = df[df['Age'].isna()]

X_train = train_data.drop('Age', axis=1)
y_train = train_data['Age']
X_test = test_data.drop('Age', axis=1)

# Обучение модели
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Предсказание возраста для строк с пропущенными значениями
predicted_ages = model.predict(X_test)

# Заполнение пропусков в оригинальном DataFrame
df.loc[df['Age'].isna(), 'Age'] = predicted_ages

# Проверка результатов
df


,Pclass,SibSp,Parch,Fare,Age,Sex_male
0,3,1,0,7.2500,22.000,True
1,1,1,0,71.2833,38.000,False
2,3,0,0,7.9250,26.000,False
3,1,1,0,53.1000,35.000,False
4,3,0,0,8.0500,35.000,True
...,...,...,...,...,...,...
886,2,0,0,13.0000,27.000,True
887,1,0,0,30.0000,19.000,False
888,3,1,2,23.4500,16.505,False
889,1,0,0,30.0000,26.000,True


In [134]:
train["Age"] = df["Age"]
train["Sex"] = df["Sex_male"]
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",True,22.000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",False,38.000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",False,26.000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",False,35.000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",True,35.000,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",True,27.000,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",False,19.000,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",False,16.505,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",True,26.000,0,0,111369,30.0000,C148,C


In [135]:
feature = ["Survived", 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare',"Age"]
corr_mat = train[feature].corr()
corr_mat

,Survived,Pclass,Sex,SibSp,Parch,Fare,Age
Survived,1.000000,-0.338481,-0.543351,-0.035322,0.081629,0.257307,-0.070299
Pclass,-0.338481,1.000000,0.131900,0.083081,0.018443,-0.549500,-0.393585
Sex,-0.543351,0.131900,1.000000,-0.114631,-0.245489,-0.182333,0.134353
SibSp,-0.035322,0.083081,-0.114631,1.000000,0.414838,0.159651,-0.316004
Parch,0.081629,0.018443,-0.245489,0.414838,1.000000,0.216225,-0.213858
Fare,0.257307,-0.549500,-0.182333,0.159651,0.216225,1.000000,0.089427
Age,-0.070299,-0.393585,0.134353,-0.316004,-0.213858,0.089427,1.000000


In [136]:
train = pd.read_csv("train.csv")
train["Age"] = df["Age"]
train["Sex"] = df["Sex_male"]
featuress = ['Pclass','Sex', 'Age', 'SibSp','Parch','Fare']
X = train[featuress]
y = train['Survived']

X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


Accuracy: 0.78


Best parameters: {'learning_rate': 0.12538077692527183, 'max_depth': 3, 'n_estimators': 118}
Best accuracy: 0.8156424581005587


In [165]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(best_model,X,y,cv=5,scoring='accuracy')
scores

array([0.81005587, 0.82022472, 0.87078652, 0.8258427 , 0.84269663])

C:\Users\casov\AppData\Local\Temp\ipykernel_5580\742301934.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test["Fare"].fillna(test["Fare"].mean(), inplace=True)


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [167]:


# Загрузка данных


# Выбор признаков (исключите 'Age' из признаков)
features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']  # Примеры признаков
df2 = pd.get_dummies(test[features + ['Age']], drop_first=True)

# Разделение данных на обучающую и тестовую выборки
train_data = df2[df2['Age'].notna()]
test_data = df2[df2['Age'].isna()]

X_train = train_data.drop('Age', axis=1)
y_train = train_data['Age']
X_test = test_data.drop('Age', axis=1)

# Обучение модели
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Предсказание возраста для строк с пропущенными значениями
predicted_ages = model.predict(X_test)

# Заполнение пропусков в оригинальном DataFrame
df2.loc[df2['Age'].isna(), 'Age'] = predicted_ages

# Проверка результатов
df2


,Pclass,SibSp,Parch,Fare,Age,Sex_male
0,3,0,0,7.8292,34.500000,True
1,3,1,0,7.0000,47.000000,False
2,2,0,0,9.6875,62.000000,True
3,3,0,0,8.6625,27.000000,True
4,3,1,1,12.2875,22.000000,False
...,...,...,...,...,...,...
413,3,0,0,8.0500,26.352706,True
414,1,0,0,108.9000,39.000000,False
415,3,0,0,7.2500,38.500000,True
416,3,0,0,8.0500,26.352706,True


In [168]:
test["Age"] = df2["Age"]
test["Sex"] = df2["Sex_male"]

In [182]:
test = test[featuress]
submission = pd.read_csv("gender_submission.csv")

lol = best_model.predict(test)
accuracy2 = accuracy_score(y_true=submission["Survived"],y_pred=lol)
accuracy2


0.8875598086124402

In [176]:
lol.sum()

137

In [175]:
submission["Survived"].sum()

152

In [183]:
137/152

0.9013157894736842

In [172]:

pred = pd.DataFrame({
    "PassengerId" : submission['PassengerId'],
    "Survived" : lol
})
pred.to_csv('result.csv', index=False)

In [198]:
dff = pd.read_csv('result.csv')
new_lol = dff["Survived"]
acc = accuracy_score(np.array(new_lol),submission["Survived"])
acc


0.8875598086124402

In [194]:
np.array(new_lol)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [193]:
pd.Series(lol)

0      0
1      0
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Length: 418, dtype: int32